<a href="https://colab.research.google.com/github/rbdus0715/information-retrieval/blob/main/boolean_retrieval/boolean_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[reference github](https://github.com/jin-zhe/boolean-retrieval-engine)</br>
[dataset](https://www.kaggle.com/datasets/boldy717/reutersnltk)

In [1]:
!unzip -qq /content/drive/MyDrive/datasets/kaggle.nltk_reuters_dataset.data/archive.zip

In [2]:
import pandas as pd

file = '/content/reutersNLTK.xlsx'
df = pd.read_excel(file, engine='openpyxl')
df = df['text']

In [3]:
!mkdir document_dir

In [8]:
import os
for index, row in df.items():
    file_name = os.path.join('document_dir', f'{index+1}')
    with open(file_name, 'w') as f:
        f.write(row)
f.close()

In [9]:
queries_file = '/content/queries_file'
posting_file = '/content/posting_file'

query = 'fear OR rais'

with open(queries_file, 'w') as f:
    f.write(query)
f.close()

In [10]:
!touch output_file

index

In [11]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import sys
import getopt
import codecs
import os
import struct
import timeit

LIMIT = None                # (for testing) to limit the number of documents indexed
IGNORE_STOPWORDS = True     # toggling the option for ignoring stopwords
IGNORE_NUMBERS = True       # toggling the option for ignoring numbers
IGNORE_SINGLES = True       # toggling the option for ignoring single character tokens
RECORD_TIME = False         # toggling for recording the time taken for indexer
BYTE_SIZE = 4               # docID is in int

document_directory = '/content/document_dir'
dictionary_file = '/content/dictionary_file'
posting_file = '/content/posting_file'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def is_number(token):
    token = token.replace(",", "")  # ignore commas in token
    # tries if token can be parsed as float
    try:
        float(token)
        return True
    except ValueError:
        return False

In [14]:
def index(document_directory, dictionary_file, posting_file):
    docID_list = [int(docID_string) for docID_string in os.listdir(document_directory)]
    docID_list.sort()

    stemmer = nltk.stem.porter.PorterStemmer()
    stopwords = nltk.corpus.stopwords.words('english')
    docs_indexed = 0
    dictionary = {}

    for docID in docID_list:
        if LIMIT and docs_indexed == LIMIT: break
        file_path = os.path.join(document_directory, str(docID))

        file = codecs.open(file_path, encoding='utf-8')
        document = file.read()                  # read entire document
        tokens = nltk.word_tokenize(document)   # list of word tokens from document

        # for each term in document
        for word in tokens:
            term = word.lower()         # casefolding
            if (IGNORE_STOPWORDS and term in stopwords):    continue    # if ignoring stopwords
            if (IGNORE_NUMBERS and is_number(term)):        continue    # if ignoring numbers
            term = stemmer.stem(term)   # stemming
            if (term[-1] == "'"):
                term = term[:-1]        # remove apostrophe
            if (IGNORE_SINGLES and len(term) == 1):         continue    # if ignoring single terms

            if (term not in dictionary):
                dictionary[term] = [docID]
            else:
                if (dictionary[term][-1] != docID):
                    dictionary[term].append(docID)

        docs_indexed += 1
        file.close()

    dict_file = codecs.open(dictionary_file, 'w', encoding='utf-8')
    post_file = open(posting_file, 'wb')

    byte_offset = 0

    # write list of docIDs indexed to first line of dictionary
    dict_file.write('Indexed from docIDs:')
    for i in range(docs_indexed):
        dict_file.write(str(docID_list[i]) + ',')
    dict_file.write('\n')

    # build dictionary file and postings file
    for term, postings_list in dictionary.items():
        df = len(postings_list)                     # document frequency is the same as length of postings list

        # write each posting into postings file
        for docID in postings_list:
            posting = struct.pack('I', docID)   # pack docID into a byte array of size 4
            post_file.write(posting)

        # write to dictionary file and update byte offset
        dict_file.write(term + " " + str(df) + " " + str(byte_offset) + "\n")
        byte_offset += BYTE_SIZE * df

    # close files
    dict_file.close()
    post_file.close()

In [15]:
index(document_directory, dictionary_file, posting_file)

search

In [16]:
import re
import nltk
import sys
import getopt
import codecs
import struct
import math
import io
import collections
import timeit

RECORD_TIME = False # toggling for recording the time taken for indexer
BYTE_SIZE = 4       # docID is in int

In [17]:
def load_dictionary(dict_file):
    dictionary = {}                 # dictionary map loaded
    indexed_docIDs = []             # list of all docIDs indexed
    docIDs_processed = False        # if indexed_docIDs is processed

    # load each term along with its df and postings file pointer to dictionary
    for entry in dict_file.read().split('\n'):
        # if entry is not empty (last line in dictionary file is empty)
        if (entry):
            # if first line of dictionary, process list of docIDs indexed
            if (not docIDs_processed):
                indexed_docIDs = [int(docID) for docID in entry[20:-1].split(',')]
                docIDs_processed = True
            # else if dictionary terms and their attributes
            else:
                token = entry.split(" ")
                term = token[0]
                df = int(token[1])
                offset = int(token[2])
                dictionary[term] = (df, offset)

    return (dictionary, indexed_docIDs)

In [18]:
def load_posting_list(post_file, length, offset):
    post_file.seek(offset)
    posting_list = []
    for i in range(length):
        posting = post_file.read(BYTE_SIZE)
        docID = struct.unpack('I', posting)[0]
        posting_list.append(docID)
    return posting_list

def shunting_yard(infix_tokens):
    # define precedences
    precedence = {}
    precedence['NOT'] = 3
    precedence['AND'] = 2
    precedence['OR'] = 1
    precedence['('] = 0
    precedence[')'] = 0

    # declare data strucures
    output = []
    operator_stack = []

    # while there are tokens to be read
    for token in infix_tokens:

        # if left bracket
        if (token == '('):
            operator_stack.append(token)

        # if right bracket, pop all operators from operator stack onto output until we hit left bracket
        elif (token == ')'):
            operator = operator_stack.pop()
            while operator != '(':
                output.append(operator)
                operator = operator_stack.pop()

        # if operator, pop operators from operator stack to queue if they are of higher precedence
        elif (token in precedence):
            # if operator stack is not empty
            if (operator_stack):
                current_operator = operator_stack[-1]
                while (operator_stack and precedence[current_operator] > precedence[token]):
                    output.append(operator_stack.pop())
                    if (operator_stack):
                        current_operator = operator_stack[-1]

            operator_stack.append(token) # add token to stack

        # else if operands, add to output list
        else:
            output.append(token.lower())

    # while there are still operators on the stack, pop them into the queue
    while (operator_stack):
        output.append(operator_stack.pop())
    # print ('postfix:', output)  # check
    return output

In [19]:
def boolean_NOT(right_operand, indexed_docIDs):
    # complement of an empty list is list of all indexed docIDs
    if (not right_operand):
        return indexed_docIDs

    result = []
    r_index = 0 # index for right operand
    for item in indexed_docIDs:
        # if item do not match that in right_operand, it belongs to compliment
        if (item != right_operand[r_index]):
            result.append(item)
        # else if item matches and r_index still can progress, advance it by 1
        elif (r_index + 1 < len(right_operand)):
            r_index += 1
    return result


def boolean_OR(left_operand, right_operand):
    result = []     # union of left and right operand
    l_index = 0     # current index in left_operand
    r_index = 0     # current index in right_operand

    # while lists have not yet been covered
    while (l_index < len(left_operand) or r_index < len(right_operand)):
        # if both list are not yet exhausted
        if (l_index < len(left_operand) and r_index < len(right_operand)):
            l_item = left_operand[l_index]  # current item in left_operand
            r_item = right_operand[r_index] # current item in right_operand

            # case 1: if items are equal, add either one to result and advance both pointers
            if (l_item == r_item):
                result.append(l_item)
                l_index += 1
                r_index += 1

            # case 2: l_item greater than r_item, add r_item and advance r_index
            elif (l_item > r_item):
                result.append(r_item)
                r_index += 1

            # case 3: l_item lower than r_item, add l_item and advance l_index
            else:
                result.append(l_item)
                l_index += 1

        # if left_operand list is exhausted, append r_item and advance r_index
        elif (l_index >= len(left_operand)):
            r_item = right_operand[r_index]
            result.append(r_item)
            r_index += 1

        # else if right_operand list is exhausted, append l_item and advance l_index
        else:
            l_item = left_operand[l_index]
            result.append(l_item)
            l_index += 1

    return result


def boolean_AND(left_operand, right_operand):
    # perform 'merge'
    result = []                                 # results list to be returned
    l_index = 0                                 # current index in left_operand
    r_index = 0                                 # current index in right_operand
    l_skip = int(math.sqrt(len(left_operand)))  # skip pointer distance for l_index
    r_skip = int(math.sqrt(len(right_operand))) # skip pointer distance for r_index

    while (l_index < len(left_operand) and r_index < len(right_operand)):
        l_item = left_operand[l_index]  # current item in left_operand
        r_item = right_operand[r_index] # current item in right_operand

        # case 1: if match
        if (l_item == r_item):
            result.append(l_item)   # add to results
            l_index += 1            # advance left index
            r_index += 1            # advance right index

        # case 2: if left item is more than right item
        elif (l_item > r_item):
            # if r_index can be skipped (if new r_index is still within range and resulting item is <= left item)
            if (r_index + r_skip < len(right_operand)) and right_operand[r_index + r_skip] <= l_item:
                r_index += r_skip
            # else advance r_index by 1
            else:
                r_index += 1

        # case 3: if left item is less than right item
        else:
            # if l_index can be skipped (if new l_index is still within range and resulting item is <= right item)
            if (l_index + l_skip < len(left_operand)) and left_operand[l_index + l_skip] <= r_item:
                l_index += l_skip
            # else advance l_index by 1
            else:
                l_index += 1

    return result

In [20]:
def process_query(query, dictionary, post_file, indexed_docIDs):
    stemmer = nltk.stem.porter.PorterStemmer() # instantiate stemmer
    # prepare query list
    query = query.replace('(', '( ')
    query = query.replace(')', ' )')
    query = query.split(' ')

    results_stack = []
    postfix_queue = collections.deque(shunting_yard(query)) # get query in postfix notation as a queue

    while postfix_queue:
        token = postfix_queue.popleft()
        result = [] # the evaluated result at each stage
        # if operand, add postings list for term to results stack
        if (token != 'AND' and token != 'OR' and token != 'NOT'):
            token = stemmer.stem(token) # stem the token
            # default empty list if not in dictionary
            if (token in dictionary):
                result = load_posting_list(post_file, dictionary[token][0], dictionary[token][1])

        # else if AND operator
        elif (token == 'AND'):
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            # print(left_operand, 'AND', left_operand) # check
            result = boolean_AND(left_operand, right_operand)   # evaluate AND

        # else if OR operator
        elif (token == 'OR'):
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            # print(left_operand, 'OR', left_operand) # check
            result = boolean_OR(left_operand, right_operand)    # evaluate OR

        # else if NOT operator
        elif (token == 'NOT'):
            right_operand = results_stack.pop()
            # print('NOT', right_operand) # check
            result = boolean_NOT(right_operand, indexed_docIDs) # evaluate NOT

        # push evaluated result back to stack
        results_stack.append(result)
        # print ('result', result) # check

    # NOTE: at this point results_stack should only have one item and it is the final result
    if len(results_stack) != 1: print ("ERROR: results_stack. Please check valid query") # check for errors

    return results_stack.pop()


In [21]:
def search(dictionary_file, postings_file, queries_file, output_file):
    # open files
    dict_file = codecs.open(dictionary_file, encoding='utf-8')
    post_file = io.open(postings_file, 'rb')
    query_file = codecs.open(queries_file, encoding='utf-8')
    out_file = open(output_file, 'w')

    # load dictionary to memory
    loaded_dict = load_dictionary(dict_file)
    dictionary = loaded_dict[0]     # dictionary map
    indexed_docIDs = loaded_dict[1] # list of all docIDs indexed in sorted order
    dict_file.close()

    # process each query
    queries_list = query_file.read().splitlines()
    for i in range(len(queries_list)):
        query = queries_list[i]
        result = process_query(query, dictionary, post_file, indexed_docIDs)
        # write each result to output
        for j in range(len(result)):
            docID = str(result[j])
            if (j != len(result) - 1):
                docID += ' '
            out_file.write(docID)
        if (i != len(queries_list) - 1):
            out_file.write('\n')

    # close files
    post_file.close()
    query_file.close()
    out_file.close()

In [22]:
queries_file = '/content/queries_file'
output_file = '/content/output_file'

search(dictionary_file, posting_file, queries_file, output_file)